This notebook demonstrates how to set up and execute a structured workflow using **CrewAI** in a Google Colab environment, leveraging LLMs to automate summarization and decision-making tasks.

In [1]:
# Install necessary packages
# The 'crewai[tools]' package provides the CrewAI framework and its associated tools.
# 'nest_asyncio' allows nested use of asyncio, which is useful in Jupyter/Colab environments.
%pip install crewai[tools]
!pip install nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.9/545.9 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.9 MB/s eta 0:

In [2]:
# Apply nest_asyncio to allow nested event loops in the notebook environment.
import nest_asyncio
nest_asyncio.apply()

In [3]:
# Import necessary modules
from crewai.flow.flow import Flow, listen, start  # CrewAI's Flow framework
from pydantic import BaseModel  # For structured state management
from google.colab import userdata  # To access user-specific data in Colab
from litellm import completion  # For interacting with language models

In [4]:
# ------------------------------
# State Management
# ------------------------------
# Define a structured state using Pydantic's BaseModel.
# This state will persist data throughout the workflow.
class WorkflowState(BaseModel):
    team_updates: str = ""  # To store team updates
    task_summary: str = ""  # To store the summarized tasks
    decision_support: str = ""  # To store decision support recommendations

In [5]:
# ------------------------------
# Orchestration Flow Class
# ------------------------------
# Define the main workflow orchestration class inheriting from CrewAI's Flow.
class LLMOrchestrationFlow(Flow[WorkflowState]):

    # Retrieve the GROQ API key from Colab's user data.
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')

    # Helper method to call the completion function with specified prompt and model.
    def call_completion(self, prompt, model):
        response = completion(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            api_key=self.GROQ_API_KEY
        )
        # Extract and return the content from the response.
        return response["choices"][0]["message"]["content"]

    # Task Automation: Summarize daily standup updates
    @start()  # This decorator marks the starting point of the workflow.
    def summarize_updates(self):
        # Create a prompt to summarize team updates.
        prompt = f"Summarize the following team updates concisely: {self.state.team_updates}"
        print("\n[MODEL 1] Generating task summary...")
        # Call the helper method with the prompt and specified model.
        result = self.call_completion(prompt, "groq/llama3-8b-8192")
        # Store the result in the structured state.
        self.state.task_summary = result
        print("\nTask Summary:")
        print(result)

    # Decision Making: Recommend project timeline adjustments
    @listen(summarize_updates)  # This decorator makes this method listen to the completion of 'summarize_updates'.
    def project_timeline_decision(self):
        # Create a prompt to get recommendations based on the task summary.
        prompt = f"Given the task summary: {self.state.task_summary}, what is your recommendation for the project timeline?"
        print("\n[MODEL 2] Generating project timeline decision...")
        # Call the helper method with the prompt and specified model.
        result = self.call_completion(prompt, "groq/mixtral-8x7b-32768")
        # Store the result in the structured state.
        self.state.decision_support = result
        print("\nProject Timeline Decision:")
        print(result)

In [6]:
# ------------------------------
# Main Execution
# ------------------------------
if __name__ == "__main__":
    # Define the team updates to be summarized.
    team_updates = """
    - Alice completed the UI design and is working on feedback integration.
    - Bob fixed the critical bug and is now optimizing backend queries.
    - Charlie is struggling with API integration and needs assistance.
    - Dave is on track with the deployment pipeline improvements.
    """
    print("\n[WORKFLOW] Starting orchestration...")
    # Initialize the workflow with the provided team updates.
    flow = LLMOrchestrationFlow(team_updates=team_updates)
    # Kick off the workflow.
    flow.kickoff()
    print("\n[WORKFLOW] Completed orchestration.")
    # Print the final state of the workflow.
    print(f"Final state:\n{flow.state}")


[WORKFLOW] Starting orchestration...
 Flow started with ID: a0b3f862-0d9e-443a-9222-145a09a9f067

[MODEL 1] Generating task summary...

Task Summary:
Here is a concise summary of the team updates:

* UI design complete, working on feedback integration (Alice)
* Critical bug fixed, optimizing backend queries (Bob)
* Struggling with API integration, needs assistance (Charlie)
* On track with deployment pipeline improvements (Dave)

[MODEL 2] Generating project timeline decision...

Project Timeline Decision:
Based on the task summary provided, here is a suggested project timeline:

1. Alice should take a few more days to integrate the feedback and finalize the UI design.
2. Bob has fixed the critical bug and is now optimizing backend queries. This work may take some time, but it is important for the overall performance of the project. Encourage Bob to continue this work, and provide any necessary support.
3. Charlie needs assistance with API integration. Offer help or assign another tea

In [9]:
from IPython.display import display, Markdown

# Convert state to a Markdown-friendly string
state_dict = flow.state.dict()  # Convert state to a dictionary
markdown_output = "### Final State:\n\n"
for key, value in state_dict.items():
    markdown_output += f"- **{key}**: {value}\n"

# Display the output as Markdown
display(Markdown(markdown_output))

### Final State:

- **id**: a0b3f862-0d9e-443a-9222-145a09a9f067
- **team_updates**: 
    - Alice completed the UI design and is working on feedback integration.
    - Bob fixed the critical bug and is now optimizing backend queries.
    - Charlie is struggling with API integration and needs assistance.
    - Dave is on track with the deployment pipeline improvements.
    
- **task_summary**: Here is a concise summary of the team updates:

* UI design complete, working on feedback integration (Alice)
* Critical bug fixed, optimizing backend queries (Bob)
* Struggling with API integration, needs assistance (Charlie)
* On track with deployment pipeline improvements (Dave)
- **decision_support**: Based on the task summary provided, here is a suggested project timeline:

1. Alice should take a few more days to integrate the feedback and finalize the UI design.
2. Bob has fixed the critical bug and is now optimizing backend queries. This work may take some time, but it is important for the overall performance of the project. Encourage Bob to continue this work, and provide any necessary support.
3. Charlie needs assistance with API integration. Offer help or assign another team member to collaborate with Charlie on this task. Allocate 3-4 days to resolve this issue.
4. Dave is on track with deployment pipeline improvements. No specific recommendations needed for this task.

Taking these factors into account, I would suggest extending the project timeline by about a week. This will allow time for Alice to complete the UI design, address Charlie's API integration concerns, and for Bob to continue his backend optimizations. Make sure to monitor progress closely and adjust the timeline if necessary.
